
# Description du Modèle et Difficultés Rencontrées

Ce notebook détaille le processus de développement d'un modèle utilisant TensorFlow et TensorFlow.js. Au cours de ce projet, plusieurs défis techniques ont dû être surmontés :

1. **Compatibilité TensorFlow.js** : Il a été nécessaire de rétrograder TensorFlow pour assurer la compatibilité avec TensorFlow.js. La version compatible devait être inférieure à TensorFlow 3.10.

2. **Problèmes avec EMNIST** : Le fichier zip de EMNIST téléchargé via `pip install` était corrompu. Une intervention manuelle dans le dossier `.cache` a été nécessaire pour corriger ce problème et permettre le bon déroulement du chargement des données.

3. **Compaptibilité avec le kernel** : Il a été nécessaire d'utiliser python 3.9 pour pouvoir être capable de travailler avec Tensorflow.js


In [ ]:
pip uninstall tensorflow tensorflowjs emnist

In [ ]:
pip install tensorflow==2.15.0

In [ ]:
pip install --upgrade tensorflowjs

In [17]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from emnist import extract_training_samples, extract_test_samples
from tensorflow.keras.utils import to_categorical
import tensorflowjs as tfjs
 
""" Obtaining the data """
x_train, y_train = extract_training_samples('byclass')
x_test, y_test = extract_test_samples('byclass')
 
""" Preprocessing """
# reshape in the format [samples][width][height][channels]
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype("float32")
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype("float32")
# normalize inputs from 0-255 to 0-1
x_train /= 255
x_test /= 255
# one hot encoding categorical outputs
y_train = to_categorical(y_train, 62)
y_test = to_categorical(y_test, 62)
 
""" Creating the model """
classifier = Sequential()
# Add model layers
classifier.add(Conv2D(64, (3, 3), input_shape=(28, 28, 1), activation="relu"))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.25))
classifier.add(Conv2D(64, (3, 3), activation="relu"))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.25))
classifier.add(Flatten())
classifier.add(Dense(units=256, activation="relu"))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=62, activation="softmax"))  # For EMNIST byclass
# Compiling the model
classifier.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Training the model
classifier.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=100, epochs=10)

# Model evaluation
scores = classifier.evaluate(x_test, y_test, verbose=0)
print("accuracy: %.2f%%"%(scores[1]*100))
print("Error: {:.2f}%".format((1-scores[1])*100))

# Save the model
tfjs.converters.save_keras_model(classifier, "model3")

Epoch 1/10
6980/6980 [==============================] - 519s 74ms/step - loss: 0.6452 - accuracy: 0.7918 - val_loss: 0.4109 - val_accuracy: 0.8516
Epoch 2/10
6980/6980 [==============================] - 510s 73ms/step - loss: 0.4800 - accuracy: 0.8342 - val_loss: 0.3798 - val_accuracy: 0.8600
Epoch 3/10
6980/6980 [==============================] - 512s 73ms/step - loss: 0.4510 - accuracy: 0.8421 - val_loss: 0.3701 - val_accuracy: 0.8625
Epoch 4/10
6980/6980 [==============================] - 517s 74ms/step - loss: 0.4371 - accuracy: 0.8455 - val_loss: 0.3654 - val_accuracy: 0.8645
Epoch 5/10
6980/6980 [==============================] - 519s 74ms/step - loss: 0.4278 - accuracy: 0.8481 - val_loss: 0.3622 - val_accuracy: 0.8642
Epoch 6/10
6980/6980 [==============================] - 516s 74ms/step - loss: 0.4218 - accuracy: 0.8498 - val_loss: 0.3583 - val_accuracy: 0.8673
Epoch 7/10
6980/6980 [==============================] - 496s 71ms/step - loss: 0.4177 - accuracy: 0.8508 - val_loss: 0

/Users/parfaitmburugu/.pyenv/versions/3.9.7/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
